In [56]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

import datetime as dt
import plotly.graph_objects as go

from tqdm import tqdm

In [2]:
data = pd.read_parquet("stock_data.parquet")
stock_names = data['comnam'].unique()

# repeat = True
# while repeat:
#     count = 0
#     dfs = []
#     sizes = []
#     relevant_stocks = np.random.choice(stock_names,size=10, replace=False)
#     for stock in relevant_stocks:
#         subset = data[data['comnam'] == stock]
#         df = subset[['prc']]
#         df.columns = [stock]
#         df.index = pd.to_datetime(subset['date'])
#         dfs.append(df)
#         sizes.append(df.shape[0])
#     dfs = [x for _,x in reversed(sorted(zip(sizes,dfs), key = lambda x:x[0]))]
#     final = dfs[0]
#     for i in range(1, 10):
#         final = final.join(dfs[i])
#     final.index.name = 'date'
#     final = final.groupby(by='date').mean()
#     final = final.interpolate()
#     period = np.random.randint(30, final.shape[0])
#     final = final.iloc[period-30:period]
#     if final.iloc[0].isna().any() or final.iloc[29].isna().any():
#         repeat = True
#     else:
#         repeat = False

In [3]:
# Remove stocks with na in price column random stocks
# TODO: Remove the complete stock if any na
data_clean = data.dropna(axis=0, subset=['prc'])

# Pick 10 stocks at random
relevant_stocks = np.random.choice(data_clean.comnam.unique(),size=10, replace=False)
data_filt = data_clean.query("comnam in @relevant_stocks")

# Pivot data (columns: stock name, index: date)
data_pivot = data_filt.pivot_table(index="date", values="prc", columns="comnam")

# Focus on window of 30 trading days with random start dt
start_dt = np.random.choice(data_pivot.index[:-30],size=1, replace=False)[0]
final = data_pivot.query("date >= @start_dt")
final = final.iloc[:30]

final

comnam,AUTODESK INC,CENTERSPACE,DIGITAL REALTY TRUST INC,ENERSYS,PIMCO DYNAMIC CR & MORT INC FD,PROSPERITY BANCSHARES INC,PRUDENTIAL REINSURANCE HOLD INC,SANMINA SCI CORP,SELIGMAN PREMIUM TECH GROWTH FD,U S X US STEEL GROUP INC
date,,,,,,,,,,
2021-03-15,273.019989,73.389999,135.070007,101.879997,22.570000,80.370003,255.970001,40.730000,30.400000,23.139999
2021-03-16,275.010010,72.599998,135.679993,101.250000,22.540001,78.769997,253.369995,41.369999,30.610001,21.309999
2021-03-17,272.839996,71.150002,134.679993,102.550003,22.490000,79.010002,251.279999,42.639999,30.900000,22.040001
2021-03-18,263.440002,72.010002,134.160004,101.250000,22.350000,79.959999,251.949997,41.669998,30.240000,22.650000
2021-03-19,261.500000,70.870003,133.509995,99.389999,22.440001,79.230003,245.830002,41.049999,30.260000,22.410000
2021-03-22,265.959991,69.360001,136.550003,97.860001,22.520000,76.540001,243.880005,41.009998,30.719999,21.790001
2021-03-23,269.000000,69.769997,140.000000,93.290001,22.290001,74.150002,241.360001,39.490002,30.370001,19.860001
2021-03-24,263.179993,68.730003,140.089996,90.790001,22.350000,73.790001,242.949997,40.000000,30.000000,19.570000
2021-03-25,262.190002,68.400002,138.360001,91.599998,22.350000,75.800003,247.860001,40.529999,29.910000,20.370001


Create an initial portfolio

In [58]:
def get_prices_window(start_dt):
    final = data_pivot.query("date >= @start_dt")
    final = final.iloc[:30]
    return final

def generate_portfolios(stock_prices, init_budget=100000):
    initial_budget = init_budget
    initial_prices = stock_prices.iloc[0].to_numpy()
    available_money = initial_budget * .95
    P_0 = np.zeros(10) #initial portfolio
    break_bool = True
    ignore =  np.random.randint(0,10)
    while break_bool:
        stock_buy = np.random.randint(0,10)
        price = initial_prices[stock_buy]
        if stock_buy != ignore:
            if (available_money * .75) - price >= 0:
                available_money = available_money - price
                P_0[stock_buy] = P_0[stock_buy] + 1
            else:
                break_bool = False

    w_0 = initial_budget * .05 + available_money
    # print(w_0)
    # print(P_0)
    final_prices = stock_prices.iloc[29].to_numpy()
    available_money = initial_budget * .9
    break_bool = True
    P_t = np.zeros(10) #target portfolio

    while break_bool:
        stock_buy = np.random.randint(0,10)
        price = final_prices[stock_buy]
        if available_money - price >= 0:
            available_money = available_money - price
            P_t[stock_buy] = P_t[stock_buy] + 1
        else:
            break_bool = False
    # print(P_t)
    return P_0, P_t, w_0, initial_prices, final_prices

# Formulations

## With Perfect Information

In [59]:
def get_naive_opt(P_0, P_t, w_0, initial_budget, initial_prices, final_prices):
    F = 5.95 * np.ones(10)# trading fees
    # P_0 = starting portfolio
    # P_t = target portfolio
    # w_0 = starting cash
    # w_f = final free cash
    budget = 20000
    M = initial_budget

    # Silence Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()

    m = gp.Model(env=env)
    # Define Variables
    P_buy = m.addMVar((10,), vtype=GRB.INTEGER)
    P_sell = m.addMVar((10,), vtype=GRB.INTEGER)
    z_sell = m.addMVar((10,), vtype=GRB.BINARY)
    z_buy = m.addMVar((10,), vtype=GRB.BINARY)
    # Constraints

    ## Meet Target
    m.addConstr(
        P_0 + P_buy - P_sell >= P_t
    )

    ## Have Money
    m.addConstr(
        w_0 + initial_prices @ P_sell - initial_prices @ P_buy - F @ z_sell - F @ z_buy >=0
    )

    ## if we sell a stock, we pay a trading price
    m.addConstr(
        M*(1-z_sell) >= P_sell
    )

    ## If we buy a stock, we pay a trading fee

    m.addConstr(
        M*(1-z_buy) >= P_buy
    )

    free_cash = w_0 + initial_prices @ P_sell - initial_prices @ P_buy
    # Set objective: Maximize end portfolio value
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(
        free_cash - F @ z_sell - F @ z_buy + final_prices @ (P_0 + P_buy - P_sell)
    )

    m.optimize()
    final_portfolio = P_0 + P_buy.x - P_sell.x
    free_cash = w_0 + initial_prices @ P_sell.x - initial_prices @ P_buy.x
    final_portfolio_value = free_cash + final_prices @ final_portfolio
    # print(f"Final Portfolio Value: {final_portfolio_value}")
    # print(f"Free Cash: {free_cash}")
    # print(f"Final Portfolio: {final_portfolio}")
    return final_portfolio_value, final_portfolio, free_cash

## Multi-stage

##

In [60]:
def get_multi_stage_opt(P_0, P_t, w_0, initial_budget, stock_price_window):   
    # Silence Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()

    F = 5.95  * np.ones(10)# trading fees
    prices = stock_price_window.interpolate().to_numpy()
    M = initial_budget
    # P_0 = starting portfolio
    # P_t = target portfolio
    # w_0 = starting cash

    m = gp.Model(env=env)

    # Define Variables
    P_buy = m.addMVar((30,10), vtype=GRB.INTEGER)
    P_sell = m.addMVar((30,10), vtype=GRB.INTEGER)
    z_sell = m.addMVar((30,10), vtype=GRB.BINARY)
    z_buy = m.addMVar((30,10), vtype=GRB.BINARY)
    P_time = m.addMVar((30,10), vtype=GRB.INTEGER)
    w = m.addMVar((30,), vtype=GRB.CONTINUOUS, lb = 0)
    # Constraints

    ## Represent Portfolios across time

    ### Time step 0
    m.addConstr(
        P_time[0,:] == P_0 + P_buy[0,:] - P_sell[0,:]
    )
    ### Rest of the time
    m.addConstrs((
        P_time[i,:] == P_time[i-1,:] - P_buy[i,:] - P_sell[i,:] for i in range(1,30)
    ))

    ### Final_target
    m.addConstr(
        P_time[29,:] >= P_t
    )

    ## Represent money
    ### First time step
    m.addConstr(
        w[0] == w_0 - prices[0,:] @ P_buy[0,:] + prices[0,:] @ P_sell[0,:] - F @ z_buy[0,:] - F @ z_sell[0,:]
    )
    ## Rest of the time
    m.addConstrs((
        w[i] == w[i-1] - prices[i,:] @ P_buy[i,:] + prices[i,:] @ P_sell[i,:] - F @ z_buy[i,:] - F @ z_sell[i,:] for i in range(1,30)
    ))

    ## Trading fees
    ### if we sell a stock, we pay a trading price
    m.addConstr(
        M*(1-z_sell) >= P_sell
    )

    ### If we buy a stock, we pay a trading fee
    m.addConstr(
        M*(1-z_buy) >= P_buy
    )

    # Maximize end Portfolio value
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(
        w[29] + prices[29,:] @ P_time[29,:]
    )
    m.optimize()
    final_portfolio = P_time[29,:].x
    free_cash = w[29].x 
    final_portfolio_value = free_cash + prices[29,:] @ final_portfolio
    # print(f"Final Portfolio Value: {final_portfolio_value}")
    # print(f"Free Cash: {free_cash}")
    # print(f"Final Portfolio: {final_portfolio}")
    return final_portfolio_value, final_portfolio, free_cash

In [62]:
simulations = 1000
rng = np.random.default_rng()
budget_vec = rng.integers(low=1000, high=10000, size=simulations)
start_dt_vec = np.random.choice(data_pivot.index[:-30], size=simulations, replace=True)
col_names = ["portf_val", "final_portf", "cash", "init_budget"]
# df_naive = pd.DataFrame(columns = )
# df_multi = pd.DataFrame(columns = ["portf_val", "final_portf", "cash"])
naive_list = list()
multi_list = list()

# Could parallelize this but fuck it
for i, budget in enumerate(tqdm(budget_vec)):
    try:
        stock_price_window = get_prices_window(start_dt_vec[i])
        P_0, P_t, w_0, initial_prices, final_prices = generate_portfolios(stock_prices = stock_price_window, init_budget=budget)
        final_portfolio_value, final_portfolio, free_cash = get_naive_opt(P_0, P_t, w_0, budget, initial_prices, final_prices)
        naive_list.append(pd.Series([final_portfolio_value, final_portfolio, free_cash, budget], index = col_names))
        final_portfolio_value, final_portfolio, free_cash = get_multi_stage_opt(P_0, P_t, w_0, budget, stock_price_window)
        multi_list.append(pd.Series([final_portfolio_value, final_portfolio, free_cash, budget], index = col_names))
    except Exception as e:
        print(e)

df_naive = pd.concat(naive_list, axis=1).T
df_multi = pd.concat(multi_list, axis=1).T

  0%|          | 3/1000 [00:01<07:02,  2.36it/s]

Double value is Nan.
Unable to retrieve attribute 'x'


  1%|          | 11/1000 [00:03<06:17,  2.62it/s]

Double value is Nan.


  2%|▏         | 15/1000 [00:05<05:49,  2.82it/s]

...

 97%|█████████▋| 973/1000 [06:06<00:10,  2.48it/s]

Double value is Nan.


100%|█████████▉| 997/1000 [06:15<00:01,  2.42it/s]

Unable to retrieve attribute 'x'
Double value is Nan.


100%|██████████| 1000/1000 [06:16<00:00,  2.66it/s]


In [63]:
df_naive

,portf_val,final_portf,cash,init_budget
0,8797.600151,"[9.0, 9.0, 4.0, 8.0, 10.0, 9.0, 7.0, 8.0, 24.0...",10.079998,9116
1,9240.049903,"[7.0, 10.0, 11.0, 13.0, 1.0, 5.0, 17.0, 8.0, 9...",1.159797,7717
2,2668.069988,"[1.0, 4.0, 3.0, 2.0, 2.0, 7.0, 3.0, 0.0, 2.0, ...",5.689983,2557
3,5448.549982,"[6.0, 9.0, 1.0, 5.0, 5.0, 3.0, 3.0, 3.0, 3.0, ...",1.81991,4768
4,3469.855116,"[5.0, 5.0, 1.0, 3.0, 6.0, 4.0, 5.0, 2.0, 5.0, ...",4.44508,2865
...,...,...,...,...
815,9400.520027,"[9.0, 4.0, 5.0, 7.0, 8.0, 8.0, 8.0, 45.0, 8.0,...",5.570091,8684
816,8503.409992,"[10.0, 19.0, 7.0, 2.0, 5.0, 7.0, 4.0, 3.0, 4.0...",14.480099,7731
817,9198.510035,"[4.0, 18.0, 9.0, 8.0, 9.0, 7.0, 11.0, 7.0, 9.0...",16.690046,9528
818,3019.34998,"[1.0, 4.0, 4.0, 3.0, 4.0, 5.0, 2.0, 22.0, 6.0,...",6.839972,2761


In [64]:
df_multi

,portf_val,final_portf,cash,init_budget
0,8843.480175,"[9.0, 9.0, 4.0, 8.0, 10.0, 9.0, 7.0, 8.0, 8.0,...",663.3200073242177,9116
1,9394.099928,"[7.0, 9.0, 11.0, 12.0, 1.0, 5.0, 7.0, 8.0, 9.0...",2462.2798671722408,7717
2,2685.209982,"[1.0, 4.0, 3.0, 2.0, 2.0, 7.0, 3.0, 0.0, 2.0, ...",504.49997329711925,2557
3,5637.909922,"[6.0, 9.0, 1.0, 5.0, 5.0, 3.0, 3.0, 3.0, 3.0, ...",1403.659860610962,4768
4,3580.315096,"[5.0, 5.0, 1.0, 3.0, 6.0, 4.0, 1.0, 2.0, 5.0, ...",1030.0850449550144,2865
...,...,...,...,...
815,9539.630186,"[9.0, 4.0, 5.0, 7.0, 8.0, 8.0, 8.0, 7.0, 8.0, ...",1716.740209579467,8684
816,8503.979992,"[10.0, 3.0, 7.0, 2.0, 5.0, 7.0, 4.0, 3.0, 4.0,...",1560.800064086914,7731
817,9272.650084,"[4.0, 12.0, 9.0, 8.0, 9.0, 7.0, 11.0, 5.0, 9.0...",767.970071792603,9528
818,3019.349981,"[1.0, 4.0, 4.0, 3.0, 4.0, 5.0, 2.0, 22.0, 6.0,...",6.839972482288049,2761


In [66]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y = df_multi.portf_val - df_naive.portf_val,
        x = df_multi.init_budget,
        mode = "markers",

    )
)

fig.update_xaxes(
    title="Starting Budget"
)

fig.update_yaxes(
    title="Multi-period Advantage"
)